In [1]:
import requests
import pandas as pd
from zipfile import ZipFile, ZIP_DEFLATED
import os

# Define the base URL for the API
base_url = "https://api.data.gov.sg/v1/environment/air-temperature"

# Define the years for which to retrieve data
years = ["2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024"]

def fetch_data_for_year(year):
    """Fetches and saves air temperature data for a given year."""
    all_readings = []
    # Loop through all months and days (simplified here, you should handle month/day iteration properly)
    for month in range(1, 13):
        for day in range(1, 32):
            date = f'{year}-{month:02d}-{day:02d}'
            response = requests.get(f"{base_url}?date={date}")
            if response.status_code == 200:
                data = response.json()
                for item in data['items']:
                    for reading in item['readings']:
                        all_readings.append({
                            'timestamp': item['timestamp'],
                            'station_id': reading['station_id'],
                            'temperature': reading['value']
                        })
            print(f"Data fetched for {date}")

    # Convert all readings into a DataFrame
    df = pd.DataFrame(all_readings)
    # Save the DataFrame to a CSV file
    csv_filename = f"AirTemperature_{year}.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Data for {year} saved to {csv_filename}")
    return csv_filename

def compress_files(filenames, archive_name):
    """Compresses a list of files into a ZIP archive."""
    with ZipFile(archive_name, 'w', ZIP_DEFLATED) as zipf:
        for file in filenames:
            zipf.write(file)
            os.remove(file)  # Optionally remove the file after compression
    print(f"Created archive {archive_name}")

# Process each year and collect filenames
filenames = [fetch_data_for_year(year) for year in years]

# Compress all CSV files into a single ZIP file
compress_files(filenames, 'AirTemperatureData.zip')

Data fetched for 2019-01-01
Data fetched for 2019-01-02
Data fetched for 2019-01-03
Data fetched for 2019-01-04
Data fetched for 2019-01-05
Data fetched for 2019-01-06
Data fetched for 2019-01-07
Data fetched for 2019-01-08
Data fetched for 2019-01-09
Data fetched for 2019-01-10
Data fetched for 2019-01-11
Data fetched for 2019-01-12
Data fetched for 2019-01-13
Data fetched for 2019-01-14
Data fetched for 2019-01-15
Data fetched for 2019-01-16
Data fetched for 2019-01-17
Data fetched for 2019-01-18
Data fetched for 2019-01-19
Data fetched for 2019-01-20
Data fetched for 2019-01-21
Data fetched for 2019-01-22
Data fetched for 2019-01-23
Data fetched for 2019-01-24
Data fetched for 2019-01-25
Data fetched for 2019-01-26
Data fetched for 2019-01-27
Data fetched for 2019-01-28
Data fetched for 2019-01-29
Data fetched for 2019-01-30
Data fetched for 2019-01-31
Data fetched for 2019-02-01
Data fetched for 2019-02-02
Data fetched for 2019-02-03
Data fetched for 2019-02-04
Data fetched for 201

In [4]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta

# Define the base URL for the API
base_url = "https://api.data.gov.sg/v1/environment/air-temperature"

def fetch_station_metadata():
    """Fetches metadata for all weather stations across all days of the year to ensure complete coverage."""
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    current_date = start_date
    station_info = {}

    while current_date <= end_date:
        date_str = current_date.strftime('%Y-%m-%d')
        response = requests.get(f"{base_url}?date={date_str}")
        if response.status_code == 200:
            data = response.json()
            stations = data['metadata']['stations']
            for station in stations:
                station_id = station['id']
                # Only update if station_id is not already in the dictionary
                if station_id not in station_info:
                    station_info[station_id] = {
                        'device_id': station['device_id'],
                        'name': station['name'],
                        'longitude': station['location']['longitude'],
                        'latitude': station['location']['latitude']
                    }
            print("Station metadata fetched for " + str(current_date))

        else:
            print(f"Failed to fetch data for {date_str}")
        current_date += timedelta(days=1)  # Increment day by one

    return station_info

def fetch_data_for_year(year, station_info):
    """Fetches and saves air temperature data and metadata for a given year."""
    all_readings = []
    # Loop through all months and days (simplified here, you should handle month/day iteration properly)
    for month in range(1, 13):
        for day in range(1, 32):
            date = f'{year}-{month:02d}-{day:02d}'
            response = requests.get(f"{base_url}?date={date}")
            if response.status_code == 200:
                data = response.json()
                for item in data['items']:
                    for reading in item['readings']:
                        station_id = reading['station_id']
                        all_readings.append({
                            'timestamp': item['timestamp'],
                            'station_id': station_id,
                            'temperature': reading['value'],
                            'device_id': station_info[station_id]['device_id'],
                            'station_name': station_info[station_id]['name'],
                            'longitude': station_info[station_id]['longitude'],
                            'latitude': station_info[station_id]['latitude']
                        })
            print(f"Data fetched for {date}")

    # Convert all readings into a DataFrame
    df = pd.DataFrame(all_readings)
    # Define the path to save the CSV file
    csv_filepath = os.path.join("C:", "LocalOneDrive", "Documents", "Desktop", "MTI", "UHI-Project", "MSE-ES-UHI", "Data", "WeatherReadings", "AirTemperature", "AirTemperature_{year}_md.csv")
    # Save the DataFrame to a CSV file at the specified location
    df.to_csv(csv_filepath, index=False)
    print(f"Data for {year} saved to {csv_filepath}")
    return csv_filepath

# Get station metadata
station_metadata = fetch_station_metadata()

# Fetch and save data for 2023
fetch_data_for_year("2023", station_metadata)

print("Completed processing and saving data.")

Station metadata fetched for 2023-01-01 00:00:00
Station metadata fetched for 2023-01-02 00:00:00
Station metadata fetched for 2023-01-03 00:00:00
Station metadata fetched for 2023-01-04 00:00:00
Station metadata fetched for 2023-01-05 00:00:00
Station metadata fetched for 2023-01-06 00:00:00
Station metadata fetched for 2023-01-07 00:00:00
Station metadata fetched for 2023-01-08 00:00:00
Station metadata fetched for 2023-01-09 00:00:00
Station metadata fetched for 2023-01-10 00:00:00
Station metadata fetched for 2023-01-11 00:00:00
Station metadata fetched for 2023-01-12 00:00:00
Station metadata fetched for 2023-01-13 00:00:00
Station metadata fetched for 2023-01-14 00:00:00
Station metadata fetched for 2023-01-15 00:00:00
Station metadata fetched for 2023-01-16 00:00:00
Station metadata fetched for 2023-01-17 00:00:00
Station metadata fetched for 2023-01-18 00:00:00
Station metadata fetched for 2023-01-19 00:00:00
Station metadata fetched for 2023-01-20 00:00:00
Station metadata fet

OSError: Cannot save file into a non-existent directory: 'C:LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\WeatherReadings\AirTemperature'

In [8]:
# Assuming 'station_info' is your dictionary containing the station metadata
def save_metadata_to_csv(station_metadata, filepath):
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame.from_dict(station_metadata, orient='index')
    
    # Save the DataFrame to a CSV file
    df.to_csv(filepath, index=False)
    print(f"Metadata saved to {filepath}")

save_metadata_to_csv(station_metadata, r"C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\WeatherReadings\AirTemperature\AirTemperature_metadata_2023.csv")

Metadata saved to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\WeatherReadings\AirTemperature\AirTemperature_metadata_2023.csv


In [1]:
import requests
import pandas as pd
from zipfile import ZipFile, ZIP_DEFLATED
import os

# Define the base URL for the API
base_url = "https://api.data.gov.sg/v1/environment/air-temperature"

# Define the years for which to retrieve data
years = ["2023", "2024"]

# Directory to save the ZIP files
data_dir = "C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\WeatherReadings"

def fetch_data_for_year(year):
    """Fetches and saves air temperature data for a given year."""
    all_readings = []
    # Loop through all months and days
    for month in range(1, 13):
        for day in range(1, 32):
            try:
                date = f'{year}-{month:02d}-{day:02d}'
                response = requests.get(f"{base_url}?date={date}")
                if response.status_code == 200:
                    data = response.json()
                    for item in data['items']:
                        for reading in item['readings']:
                            all_readings.append({
                                'timestamp': item['timestamp'],
                                'station_id': reading['station_id'],
                                'temperature': reading['value']
                            })
                print(f"Data fetched for {date}")
            except Exception as e:
                print(f"Failed to fetch data for {date}: {e}")

    # Convert all readings into a DataFrame
    df = pd.DataFrame(all_readings)
    # Save the DataFrame to a CSV file
    csv_filename = os.path.join(data_dir, f"AirTemperature_{year}.csv")
    df.to_csv(csv_filename, index=False)
    print(f"Data for {year} saved to {csv_filename}")
    return csv_filename

# def compress_files(filenames, year):
#     """Compresses a list of files into a ZIP archive for the specified year."""
#     zip_filename = os.path.join(data_dir, f"AirTemperature_{year}.zip")
#     with ZipFile(zip_filename, 'w', ZIP_DEFLATED) as zipf:
#         for file in filenames:
#             zipf.write(file)
#             os.remove(file)  # Optionally remove the file after compression
#     print(f"Created archive {zip_filename}")

def compress_files(filenames, archive_name):
    """Compresses a list of files into a ZIP archive."""
    with ZipFile(archive_name, 'w', ZIP_DEFLATED) as zipf:
        for file in filenames:
            # Extract the filename and use it as the name in the zip
            zipf.write(file, os.path.basename(file))
            os.remove(file)  # Optionally remove the file after compression
    print(f"Created archive {archive_name}")

# Process data for each year
for year in years:
    filename = fetch_data_for_year(year)
    compress_files([filename], year)

Data fetched for 2023-01-01
Data fetched for 2023-01-02
Data fetched for 2023-01-03
Data fetched for 2023-01-04
Data fetched for 2023-01-05
Data fetched for 2023-01-06
Data fetched for 2023-01-07
Data fetched for 2023-01-08
Data fetched for 2023-01-09
Data fetched for 2023-01-10
Data fetched for 2023-01-11
Data fetched for 2023-01-12
Data fetched for 2023-01-13
Data fetched for 2023-01-14
Data fetched for 2023-01-15
Data fetched for 2023-01-16
Data fetched for 2023-01-17
Data fetched for 2023-01-18
Data fetched for 2023-01-19
Data fetched for 2023-01-20
Data fetched for 2023-01-21
Data fetched for 2023-01-22
Data fetched for 2023-01-23
Data fetched for 2023-01-24
Data fetched for 2023-01-25
Data fetched for 2023-01-26
Data fetched for 2023-01-27
Data fetched for 2023-01-28
Data fetched for 2023-01-29
Data fetched for 2023-01-30
Data fetched for 2023-01-31
Data fetched for 2023-02-01
Data fetched for 2023-02-02
Data fetched for 2023-02-03
Data fetched for 2023-02-04
Data fetched for 202

KeyboardInterrupt: 

In [17]:
import requests
import pandas as pd
from zipfile import ZipFile, ZIP_DEFLATED
import os

# Define the base URL for the API
base_url = "https://api.data.gov.sg/v1/environment/wind-speed"

# Define the years for which to retrieve data
years = ["2021", "2022", "2023", "2024"]
#years = ["2014", "2015", "2016", "2017", "2018", "2019", "2020"]

# Directory to save the ZIP files
data_dir = "C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\WeatherReadings"

def fetch_data_for_year(year):
    """Fetches and saves air temperature data for a given year."""
    all_readings = []
    # Loop through all months and days
    for month in range(1, 13):
        for day in range(1, 32):
            try:
                date = f'{year}-{month:02d}-{day:02d}'
                response = requests.get(f"{base_url}?date={date}")
                if response.status_code == 200:
                    data = response.json()
                    for item in data['items']:
                        for reading in item['readings']:
                            all_readings.append({
                                'timestamp': item['timestamp'],
                                'station_id': reading['station_id'],
                                'temperature': reading['value']
                            })
                print(f"Data fetched for {date}")
            except Exception as e:
                print(f"Failed to fetch data for {date}: {e}")

    # Convert all readings into a DataFrame
    df = pd.DataFrame(all_readings)
    # Save the DataFrame to a CSV file
    csv_filename = os.path.join(data_dir, f"WindSpeed_{year}.csv")
    df.to_csv(csv_filename, index=False)
    print(f"Data for {year} saved to {csv_filename}")
    return csv_filename

def compress_files(filenames, year):
    """Compresses a list of files into a ZIP archive for the specified year."""
    zip_filename = os.path.join(data_dir, f"WindSpeed_{year}.zip")
    with ZipFile(zip_filename, 'w', ZIP_DEFLATED) as zipf:
        for file in filenames:
            zipf.write(file)
            os.remove(file)  # Optionally remove the file after compression
    print(f"Created archive {zip_filename}")

# Process data for each year
for year in years:
    filename = fetch_data_for_year(year)
    compress_files([filename], year)

Data fetched for 2021-01-01
Data fetched for 2021-01-02
Data fetched for 2021-01-03
Data fetched for 2021-01-04
Data fetched for 2021-01-05
Data fetched for 2021-01-06
Data fetched for 2021-01-07
Data fetched for 2021-01-08
Data fetched for 2021-01-09
Data fetched for 2021-01-10
Data fetched for 2021-01-11
Data fetched for 2021-01-12
Data fetched for 2021-01-13
Data fetched for 2021-01-14
Failed to fetch data for 2021-01-15: 'value'
Data fetched for 2021-01-16
Data fetched for 2021-01-17
Data fetched for 2021-01-18
Data fetched for 2021-01-19
Data fetched for 2021-01-20
Data fetched for 2021-01-21
Data fetched for 2021-01-22
Data fetched for 2021-01-23
Data fetched for 2021-01-24
Data fetched for 2021-01-25
Failed to fetch data for 2021-01-26: 'value'
Data fetched for 2021-01-27
Data fetched for 2021-01-28
Data fetched for 2021-01-29
Data fetched for 2021-01-30
Data fetched for 2021-01-31
Data fetched for 2021-02-01
Data fetched for 2021-02-02
Data fetched for 2021-02-03
Failed to fetc

In [18]:
import requests
import pandas as pd

def fetch_station_data():
    """Fetches and saves the geographical data of weather stations."""
    url = "https://api.data.gov.sg/v1/environment/air-temperature"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        stations = data['metadata']['stations']
        station_data = [{
            'station_id': station['id'],
            'name': station['name'],
            'latitude': station['location']['latitude'],
            'longitude': station['location']['longitude']
        } for station in stations]
        
        # Convert list of station data to DataFrame
        df = pd.DataFrame(station_data)
        
        # Save the DataFrame to a CSV file
        csv_filename = "WeatherStations.csv"
        df.to_csv(csv_filename, index=False)
        print(f"Weather station data saved to {csv_filename}")

fetch_station_data()

Weather station data saved to WeatherStations.csv


# Compressing file paths